# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 




In [2]:
import dask.dataframe as dd

/var/folders/md/p68hj1_91h18wtsdq4ct77gm0000gn/T/ipykernel_68695/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

#There is no Adj_Close field in the data so I created lag only for Close field.


/var/folders/md/p68hj1_91h18wtsdq4ct77gm0000gn/T/ipykernel_68695/3983797629.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
dd_feat_df = dd_feat.compute()



In [20]:
rm = dd_feat_df.groupby('ticker')['returns'].rolling(10).mean()
#resetting indexes so I can add rolling mean column with original data 
rm2 = rm.reset_index(drop=True)
dd_feat_df2 = dd_feat_df.reset_index()
#add rolling mean field
dd_feat_df2['rollingmean'] = rm2
dd_feat_df2

,ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag,returns,hi_lo_range,rollingmean
0,A,2013-12-02 00:00:00-05:00,34.875487,35.005933,34.640679,34.705902,2039962,0.0,0.0,2013,NaN,NaN,0.365255,NaN
1,A,2013-12-03 00:00:00-05:00,34.516744,34.679807,34.281936,34.523266,3462706,0.0,0.0,2013,34.705902,-0.005262,0.397871,NaN
2,A,2013-12-04 00:00:00-05:00,34.464560,35.110283,34.386293,34.947220,3377288,0.0,0.0,2013,34.523266,0.012280,0.723990,NaN
3,A,2013-12-05 00:00:00-05:00,34.797214,35.090723,34.647198,34.895050,2530939,0.0,0.0,2013,34.947220,-0.001493,0.443525,NaN
4,A,2013-12-06 00:00:00-05:00,35.064625,35.827747,35.064625,35.762524,4268513,0.0,0.0,2013,34.895050,0.024860,0.763122,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118438,YUM,2024-01-25 00:00:00-05:00,127.913548,128.386458,126.642582,128.297791,1871700,0.0,0.0,2024,127.529289,0.006026,1.743877,0.000828
118439,YUM,2024-01-26 00:00:00-05:00,128.002206,128.760844,126.770657,127.184456,1157000,0.0,0.0,2024,128.297791,-0.008678,1.990186,0.000115
118440,YUM,2024-01-29 00:00:00-05:00,127.194317,128.622910,126.396261,128.622910,1482100,0.0,0.0,2024,127.184456,0.011310,2.226649,0.001145
118441,YUM,2024-01-30 00:00:00-05:00,128.435723,129.283029,127.667238,128.691879,1421500,0.0,0.0,2024,128.622910,0.000536,1.615791,0.000672


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

Yes. As using a groupby with rolling mean in dask returns an error message - 
ValueError: Only time indexes are supported for rolling groupbys in dask dataframe. ``window`` must be a ``freq`` (e.g. '1H').

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.